In [1]:
from __future__ import division
from pyomo.environ import *
import numpy as np
import pandas as pd
import copy
from itertools import combinations as cmb
import time


In [2]:
# Two Products, 3 weeks


Base_Price = [[10,10,10],[10,10,10]]
Intercepts = [23,20]

P1_EDLP_coef = [-6,1.2]
P2_EDLP_coef = [1,-5]

P1_TPR_coef = [0.01,-0.004]
P2_TPR_coef = [-0.003,0.02]

Target_Trade_Spend = [276860,223573]
Target_EDLP_Spend = [63366,35525]
Target_TPR_Spend = [213494,188048]

Model.Locations = range(3)

No_Of_Products = 2
No_Of_Weeks = 3

Total = No_Of_Products * No_Of_Weeks

Init_List = list(set(cmb([0,0,0,1,1,1],3)))
Model.Location = range(3)

LB = [9.5,9.5,9.5,9.5,9.5,9.5,5,5,5,5,5,5,0,0,0,0,0,0]
UB = [10,10,10,10,10,10,50,50,50,50,50,50,1,1,1,1,1,1]


In [3]:

def Initialize_Fnc(Model,i):
    return Init_List[i]

# i --> range(6) as iterations
# Variables[i]                  --> EDLP (index 0,1,..5) 
# Variables[i + No_Of_Weeks*No_of_Products]    --> TPR  (index 6,7..11)
# Variables[i + 2*No_of_Products]  --> FLAG (index 12,13,..17)

def Unit_Sales_Price_Fnc(Variables):
        
    Unit_Sales = []
    Sales_Price= []

    for j in range(No_Of_Products):

        Unit_Sales.append([exp(((log(Variables[i])*P1_EDLP_coef[j])* Variables[i+2*No_Of_Products*No_Of_Weeks])
                               +((log(Base_Price[j][i])*P1_EDLP_coef[j]+
                                  Variables[i+No_Of_Products*No_Of_Weeks]*P1_TPR_coef[j]) * 
                                 (1- Variables[i+2*No_Of_Products*No_Of_Weeks]))
                               +((log(Variables[i+No_Of_Weeks])*P2_EDLP_coef[j])* 
                                 Variables[i+2*No_Of_Products*No_Of_Weeks+No_Of_Weeks])
                               +((log(Base_Price[j][i])*P2_EDLP_coef[j]+ Variables[i+No_Of_Products*No_Of_Weeks+
                                                                                   No_Of_Weeks]*
             P2_TPR_coef[j])*(1- Variables[i+2*No_Of_Products*No_Of_Weeks+No_Of_Weeks]))+Intercepts[j]) for i in Model.Locations])

        Sales_Price.append([ (Base_Price[j][i]* (1-Variables[i + No_Of_Weeks*No_Of_Products+j*No_Of_Weeks]/100) *
                         (1-Variables[i + 2*No_Of_Weeks*No_Of_Products+j*No_Of_Weeks]))+(Variables[i+j*No_Of_Weeks] * 
            Variables[i + 2*No_Of_Weeks*No_Of_Products+j*No_Of_Weeks]) for i in Model.Locations])

    return Unit_Sales,Sales_Price
            
def Dollar_Sales_Fnc(Variables):
    
    Dollar_Sales=[]
    
    Unit_Sales, Price = Unit_Sales_Price_Fnc(Variables)
    
    for j in range(No_Of_Products):
    
        Dollar_Sales_Single = [Unit_Sales[j][i]*Price[j][i] for i in Model.Locations]
        
        Dollar_Sales.append(Dollar_Sales_Single)
   
    return Dollar_Sales


def Actual_Total_Trade_Spent_Fnc(Variables):
   
    Unit_Sales, Sales_Price = Unit_Sales_Price_Fnc(Variables)
    
    Actual_Total_Trade_Spent = []
    
    for j in range(No_Of_Products):
    
        Trade_Spent = [Base_Price[j][i]-Sales_Price[j][i] for i in Model.Locations]

        Actual_Total_Trade_Spent.append([Trade_Spent[i] * Unit_Sales[j][i] for i in Model.Locations])    

    return Actual_Total_Trade_Spent

def Actual_EDLP_Trade_Spent_Fnc(Variables):
    
    Unit_Sales, Price = Unit_Sales_Price_Fnc(Variables)
    
    Actual_Total_Trade_Spent = Actual_Total_Trade_Spent_Fnc(Variables)
    
    Actual_EDLP_Trade_Spent = []
    
    for j in range(No_Of_Products):
        
        Actual_EDLP_Trade_Spent.append([Actual_Total_Trade_Spent[j][i] * 
                                        Variables[i + 2*No_Of_Weeks*No_Of_Products+j*No_Of_Weeks] for i in Model.Locations])
   
    return Actual_EDLP_Trade_Spent

def Actual_TPR_Trade_Spent_Fnc(Variables):
    
    Unit_Sales, Sales_Price = Unit_Sales_Fnc(Variables), Sales_Price_Fnc(Variables)
    
    Actual_TPR_Trade_Spent = []

    Actual_Total_Trade_Spent = Actual_Total_Trade_Spent(Variables)
    
    for j in range(No_Of_Products):
        
        Actual_TPR_Trade_Spent.append([Actual_Total_Trade_Spent[j][i] * 
                                       (1 - Variables[i+2*No_Of_Weeks*No_Of_Weeks+j*No_Of_Weeks]) for i in Model.Locations])
   
    return Actual_TPR_Trade_Spent



In [4]:

Model = ConcreteModel()

Model.Slot = Set(initialize=list(range(18)))

# Binary values for FLAG and for the rest NonNegativeReals

def variable_Domain_Set_Fnc(Model,i):

    if i < 2 * No_Of_Weeks * No_Of_Products:
    
        return NonNegativeReals
    
    else:
        
        return Binary

def Boundary_Set_Fnc(Model,i):
    
    return (LB[i],UB[i])

init=(0,1,0,1,0,1)
def Variable_Initialization_Fnc(Model,i,m=init):
    
    m,n,o,p,q,r = init
    
    if i < No_Of_Weeks * No_Of_Products: # EDLP --> initialization as Upper Boundary
    
        return 10   
    
    if i < 2 * No_Of_Weeks * No_Of_Products: # TPR --> initialization as lower boundary 
            
        return 5
    
    elif i == 12:  # FLAG --> initialization as  Binary
    
        return m

    elif i == 13:  # FLAG --> initialization as  Binary
    
        return n

    elif i == 14:  # FLAG --> initialization as  Binary
    
        return o

    elif i == 15:  # FLAG --> initialization as  Binary
    
        return p

    elif i == 16:  # FLAG --> initialization as  Binary
    
        return q

    elif i == 17:  # FLAG --> initialization as  Binary
    
        return r

init=(1,1,1,1,1,1)


combo = list(set(cmb([0,0,0,1,1,1],3)))
combo.sort()

In [5]:
import re
def get_info_from_results(results, info_string):
    i = str(results).lower().find(info_string.lower()) + len(info_string)
    value = ''
    while str(results)[i] != '\n':
        value = value + str(results)[i]
        i += 1
    return float(value)
def get_obj_frm_res(results, info_string):
#     print(results)
    results = results.lower()
    info_string = info_string.lower()
    z = re.search(info_string, results).start()
    
    return(float(results[z+109:z+125]))

In [7]:
for name in ('bonmin','ipopt'):
    time_lst = []
    obj_lst = []
    time_lst_1 = []
    
    for comb in combo:
        for com in combo:
            init=comb+com
                
            
            Model.Variables = Var(Model.Slot, within=variable_Domain_Set_Fnc, bounds=Boundary_Set_Fnc, 
                                  initialize= Variable_Initialization_Fnc)


            Dollar_Sales = Dollar_Sales_Fnc(Model.Variables)

            Total_Trade_Spent = Actual_Total_Trade_Spent_Fnc(Model.Variables)

            EDLP_Trade_Spent = Actual_EDLP_Trade_Spent_Fnc(Model.Variables)

            Model.Obj = Objective(expr=sum(sum(Dollar_Sales[i]) for i in range(len(Dollar_Sales))) ,sense= maximize)

            for i in range(No_Of_Products): # Constraints for the Products

                exec(f"Model.OverAll_P{i+1} = Constraint(expr = sum(Total_Trade_Spent[{i}]) == Target_Trade_Spend[{i}])")
                exec(f"Model.EDLP_P{i+1} = Constraint(expr = sum(EDLP_Trade_Spent[{i}]) == Target_EDLP_Spend[{i}])" )

            start_time = time.time()
            opt_1 = SolverFactory(name)
            results = opt_1.solve(Model)
            t = get_info_from_results(results, 'Time: ')
            
            end_time = time.time()
            print(f"\n SOLVER Name : {name} \n")
            print(f"\n P1 Initialization Value: {comb} , P2 Initialization value: {com} \n ")
            
            Model.display('hola.txt')
            file =  open('hola.txt', 'r')
            data = file.read()
            o = get_obj_frm_res(data, 'Objectives')
            file.close()
            
#             print("Objective: ", o)
#             print("The Elapsed Time is " , (end_time - start_time))
#             print("The Solver Time is ", t)
#             print('-------------------------------------------------------------')
            obj_lst.append(o)
            time_lst.append(t)
            time_lst_1.append(end_time - start_time)
    print(obj_lst)
    print(time_lst)
    print(time_lst_1)

    'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.objective.SimpleObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.SimpleObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>).


 SOLVER Name : bonmin 


 P1 Initialization Value: (0, 0, 0) , P2 Initialization value: (1, 1, 1) 
 
    'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.objective.SimpleObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.SimpleObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on bl


 SOLVER Name : bonmin 


 P1 Initialization Value: (0, 0, 1) , P2 Initialization value: (1, 1, 1) 
 
    'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.objective.SimpleObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.SimpleObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on bl


 SOLVER Name : bonmin 


 P1 Initialization Value: (0, 1, 1) , P2 Initialization value: (1, 1, 1) 
 
    'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.objective.SimpleObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.SimpleObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on bl


 SOLVER Name : bonmin 


 P1 Initialization Value: (1, 1, 1) , P2 Initialization value: (1, 1, 1) 
 
[566396.813079901, 566870.584721849, 566870.584721851, 566870.584721851, 566870.58472185, 566870.584721852, 565275.604137003, 566870.58472185, 575640.386593688, 576198.81760316, 566870.584721849, 576198.817603159, 576198.817603159, 566870.584721848, 576198.817603159, 566396.813079901]
[0.10378527641296387, 0.09902548789978027, 0.3599886894226074, 0.12307024002075195, 0.2881643772125244, 0.14439725875854492, 0.07386279106140137, 0.4504048824310303, 0.4397237300872803, 0.05523562431335449, 0.31981921195983887, 0.2617683410644531, 0.4160494804382324, 0.4114413261413574, 0.4241600036621094, 0.13611841201782227]
[0.12423491477966309, 0.11972188949584961, 0.38114356994628906, 0.14295029640197754, 0.30810022354125977, 0.16440224647521973, 0.09240984916687012, 0.4686403274536133, 0.459209680557251, 0.07490944862365723, 0.3390381336212158, 0.2810962200164795, 0.43534278869628906, 0.431557416915


 SOLVER Name : ipopt 


 P1 Initialization Value: (0, 0, 0) , P2 Initialization value: (1, 1, 1) 
 
    'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.objective.SimpleObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.SimpleObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on blo


 SOLVER Name : ipopt 


 P1 Initialization Value: (0, 0, 1) , P2 Initialization value: (1, 1, 1) 
 
    'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.objective.SimpleObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.SimpleObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on blo


 SOLVER Name : ipopt 


 P1 Initialization Value: (0, 1, 1) , P2 Initialization value: (1, 1, 1) 
 
    'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
    (type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.objective.SimpleObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.SimpleObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on blo


 SOLVER Name : ipopt 


 P1 Initialization Value: (1, 1, 1) , P2 Initialization value: (1, 1, 1) 
 
[448327.060115118, 567832.482714953, 564142.871351527, 566396.850059822, 567811.255708767, 566966.635897756, 566870.623005981, 566870.622993194, 567811.255708764, 576198.874731149, 561745.960712347, 575328.228800095, 567811.255720277, 566396.850033643, 564144.283913715, 566396.850015926]
[0.09206604957580566, 0.21580910682678223, 0.31771063804626465, 0.2135756015777588, 0.14360642433166504, 0.15181708335876465, 0.15068674087524414, 0.23882174491882324, 0.18978404998779297, 0.0769815444946289, 0.07534122467041016, 0.5317721366882324, 0.237168550491333, 0.145857572555542, 0.1387004852294922, 0.17841267585754395]
[0.11206436157226562, 0.23459196090698242, 0.3370523452758789, 0.23230981826782227, 0.161088228225708, 0.17009377479553223, 0.16846609115600586, 0.25636887550354004, 0.2080996036529541, 0.09415960311889648, 0.09306073188781738, 0.5504100322723389, 0.25610852241516113, 0.1644046306

In [11]:
a = [448327.060115118, 567832.482714953, 564142.871351527, 566396.850059822, 567811.255708767, 566966.635897756, 566870.623005981, 566870.622993194, 567811.255708764, 576198.874731149, 561745.960712347, 575328.228800095, 567811.255720277, 566396.850033643, 564144.283913715, 566396.850015926]
b = [0.09206604957580566, 0.21580910682678223, 0.31771063804626465, 0.2135756015777588, 0.14360642433166504, 0.15181708335876465, 0.15068674087524414, 0.23882174491882324, 0.18978404998779297, 0.0769815444946289, 0.07534122467041016, 0.5317721366882324, 0.237168550491333, 0.145857572555542, 0.1387004852294922, 0.17841267585754395]
c = [0.11206436157226562, 0.23459196090698242, 0.3370523452758789, 0.23230981826782227, 0.161088228225708, 0.17009377479553223, 0.16846609115600586, 0.25636887550354004, 0.2080996036529541, 0.09415960311889648, 0.09306073188781738, 0.5504100322723389, 0.25610852241516113, 0.16440463066101074, 0.15627002716064453, 0.19707059860229492]
pd.DataFrame(data=[a,b,c]).to_csv('out_2.csv')
!cp out_2.csv /mnt/d/1/